In [1]:
import re
import os
import time
import sqlite3
import warnings
import numpy as np
import pandas as pd 


import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.svm import LinearSVC #, SVC
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn import metrics

warnings.filterwarnings("ignore")

In [2]:
# Italian Datasets 
####################
# HaSpeeDe: http://www.di.unito.it/~tutreeb/haspeede-evalita18/index.html
TW_train = './datasets/HaSpeeDe/haspeede_TW-train.tsv'
FB_train = './datasets/HaSpeeDe/haspeede_FB-train.tsv'
TW_test = './datasets/HaSpeeDe/haspeede_TW-test.tsv'
FB_test = './datasets/HaSpeeDe/haspeede_FB-test.tsv'
# HSC: https://github.com/aitor-garcia-p/hate-speech-dataset
HS_text = './datasets/HSC_with_text.csv'

# English Datasets
####################
# StormfrontWS: https://github.com/aitor-garcia-p/hate-speech-dataset
forum_post_eng = './datasets/StormfrontWS'
# Davidson: https://github.com/t-davidson/hate-speech-and-offensive-language
davidson_eng = './datasets/Davidson_hate_speech.csv'

In [3]:
# paper here    https://www.rug.nl/research/portal/files/74389682/paper40.pdf
# download here https://drive.google.com/drive/folders/133EPm4mO9dN6A0Cw6A6Sx1ABa-25BI8e
embedding_ita1_file = './embeddings/model_hate_300.bin' 
# paper here    http://ceur-ws.org/Vol-2263/paper013.pdf
# download here http://www.italianlp.it/resources/italian-word-embeddings/
embedding_ita2_file = './embeddings/itwac128.sqlite' 
# paper here    http://ceur-ws.org/Vol-1404/paper_11.pdf
# download here http://hlt.isti.cnr.it/wordembeddings/
embedding_ita3_file = './embeddings/wiki_w2v/wiki_iter=5_algorithm=skipgram_window=10_size=300_neg-samples=10.m'
# paper here    https://arxiv.org/pdf/1301.3781.pdf
# download here https://code.google.com/archive/p/word2vec/
embedding_eng1_file = './embeddings/GoogleNews-vectors-negative300.bin.gz'
# paper here    https://nlp.stanford.edu/pubs/glove.pdf
# download here https://nlp.stanford.edu/projects/glove/
embedding_eng2_file = './embeddings/glove.6B.300d.txt'


# Gender neutral built datasets: 
bias_samples_ita = './support_files/bias_ita_libs/dataset_bias_test.csv'
bias_samples_eng = './support_files/bias_eng_libs/dataset_bias_test.csv'

## Functions to get files and embeddings

In [4]:
def get_haspeed_file(file, X, Y):
    # Load train data from the FB file
    samples, labels = [],[]

    # Load train data from the TW file
    with open(file,'r', encoding='utf-8') as fi:
        for line in fi:
            data = line.strip().split('\t')
            # get sample
            samples.append(data[1])
            # get labels
            labels.append(data[2])
    X += samples
    Y += labels
    return X, Y

def get_StormfrontWS_files(directory, mode='train'):
    all_files = []
    if mode == 'train': 
        all_files = [os.path.join(directory, 'sampled_train', file) 
                     for file in os.listdir(os.path.join(directory, 'sampled_train'))]
    if mode == 'test':
        all_files = [os.path.join(directory, 'sampled_test', file) for file in 
                     os.listdir(os.path.join(directory, 'sampled_test'))]
    if mode == 'all': 
        all_files = [os.path.join(directory, 'all_files', file)  
                     for file in os.listdir(os.path.join(directory, 'all_files'))]
    
    samples = []
    labels = []
    annotations = pd.read_csv(os.path.join(directory, 'annotations_metadata.csv'))
    for file in all_files:
        with open(file,'r', encoding='utf-8') as fi:
            samples.append(fi.readlines()[0])
        file_id = file.split('/')[-1].split('.')[0]
        if annotations[annotations['file_id'] == file_id]['label'].iloc[0] == 'noHate':
            labels.append('0')
        else:
            labels.append('1')

    return samples, labels 

def get_HSC_file(file): 
    samples, labels = [],[]
    lines = pd.read_csv(file, encoding='utf-8')
    for _, line in lines.iterrows():
        # get sample
        samples.append(line['text'])
        # get labels
        if line['hate speech'] == 'no': 
            labels.append('0')
        else: 
            labels.append('1')
    return samples, labels

def get_davidson_file(file): 
    samples, labels = [],[]
    lines = pd.read_csv(file, encoding='utf-8')
    for _, line in lines.iterrows():
        # get sample
        samples.append(line['tweet'])
        # get labels
        if line['class'] == 2:
            labels.append('0')
        else: 
            labels.append('1')
    return samples, labels

In [5]:
def get_embedding(embedding_file): 
    embeds = KeyedVectors.load(embedding_file).wv
    vocab = {word for word in embeds.index2word}
    return embeds, vocab 


def get_embedding2(embedding_file): 
    return Word2Vec.load(embedding_file).wv

def load_wordvec_model(modelFile, flagBin):
    model = KeyedVectors.load_word2vec_format(os.path.join(modelFile), binary=flagBin)
    return model

def get_embedding_sqlite(input_file):
    embedding = dict()
    #outfile = codecs.open(sys.argv[2], 'w', 'utf-8')
    with  sqlite3.connect(input_file) as conn:
        c = conn.cursor()
        for row in c.execute("SELECT * from store"):
            if row[0] == "\t":
                continue
            embedding[row[0]] = row[1:-1]
    return embedding

def load_glove_embeddings(filename):
    with open(embedding_eng2_file, 'r') as f_embed:
        vocab = [line.split()[0] for line in f_embed]
    with open(embedding_eng2_file, 'r') as f_embed:
        vec = [ [float(elem) for elem in line.split()[1:]] for line in f_embed]
    embedding = {w: v for w, v in zip(vocab, vec)}
    return embedding

## Function cleaning text

In [6]:
def cleanup_italian_text(samples):
    new_samples = []
    for string in samples:
        string = re.sub(r'@\S+','User', string)
        string = re.sub(r'http\S+', '', string)
        string = re.sub(r'\|LBR\|', '', string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\s{2,}", " ", string)
        string = re.sub(r"'", " ' ", string)
        string = string.replace("/"," ")
        string = string.replace("-"," ")

        string = string.replace("%"," percento ")
        #string = re.sub(r"[^A-Za-z0-9(),!?èéàòùì\'\`]", " ", string)
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('italian')) + r')\b\s*')
        string = pattern.sub('', string)
        string = string.strip().lower()
        new_samples.append(string)
    return new_samples

def cleanup_english_text(samples):
    new_samples = []
    for string in samples:
        string = re.sub(r'@\S+','User', string)
        string = re.sub(r'http\S+', '', string)
        string = re.sub(r'\|LBR\|', '', string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\s{2,}", " ", string)
        string = string.replace("/"," ")

        string = string.replace("'s", " ")
        string = string.replace("n't", " not ")
        string = string.replace("'ve", " have ")
        string = string.replace("'re", " are ")
        string = string.replace("I'm"," I am ")
        string = string.replace("you're"," you are ")
        string = string.replace("You're"," You are ")

        string = string.replace("%"," percent ")
        #string = re.sub(r'[^a-zA-Z0-9 ]', '', string)
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        string = pattern.sub('', string)
        string = string.strip().lower()
        new_samples.append(string)
    return new_samples

In [7]:
def get_metrics(Ytest, Yguess): 
    scores = metrics.precision_recall_fscore_support(Ytest, Yguess, average='binary', pos_label='1')
    print('precision: {}'.format(scores[0]))
    print('Recall:    {}'.format(scores[1]))
    print('F1-score:  {}'.format(scores[2]))

def get_confusion_metrics(Ytest, Yguess):
    conf_mat = metrics.confusion_matrix(Ytest, Yguess)
    print('\n Confusion Matrix')
    print(' pred \ true | nohate | hate ')
    print('      -----------------------')
    print('      nohate |   {}  | {} '.format(conf_mat[0][0],conf_mat[0][1]))
    print('        hate |   {}  | {} '.format(conf_mat[1][0],conf_mat[1][1]))
    
def get_gender_bias_metrics(bias_test_df): 
    print('Metrics related to full dataset')
    tot_pos = bias_test_df[bias_test_df['correct'] == 0]
    Yfull = bias_test_df['correct'].tolist() 
    Ypred = bias_test_df['pred'].tolist() 
    get_metrics(Yfull, Ypred)
    tp, fp, fn, tn = metrics.confusion_matrix(Yfull, Ypred).ravel()
    fpr = fp / (tp + fp)
    fnr = fn / (tn + fn)
    
    print('\n\nMetrics related to male dataset')
    Ymale = bias_test_df[bias_test_df['gender'] == 'male']['correct'].tolist() 
    Ypred = bias_test_df[bias_test_df['gender'] == 'male']['pred'].tolist() 
    get_metrics(Ymale, Ypred)
    tp_m, fp_m, fn_m, tn_m = metrics.confusion_matrix(Ymale, Ypred).ravel()
    fpr_m = fp_m / (tp_m + fp_m)
    fnr_m = fn_m / (tn_m + fn_m)
    
    print('\n\nMetrics related to female dataset')
    Yfemale = bias_test_df[bias_test_df['gender'] == 'female']['correct'].tolist() 
    Ypred = bias_test_df[bias_test_df['gender'] == 'female']['pred'].tolist()
    get_metrics(Yfemale, Ypred)
    tp_f, fp_f, fn_f, tn_f = metrics.confusion_matrix(Yfemale, Ypred).ravel()
    fpr_f = fp_f / (tp_f + fp_f)
    fnr_f = fn_f / (tn_f + fn_f)
    
    print('\n\nGender bias metrics')
    print(' False Positive Equality Difference (FPED): {}'.format(abs(fpr - fpr_m) + abs(fpr - fpr_f)))
    print(' False Negative Equality Difference (FNED): {}'.format(abs(fnr - fnr_m) + abs(fnr - fnr_f)))

## Class to apply embedding

In [8]:
class Embeddings(TransformerMixin):
    '''Transformer object turning a sentence into a single embedding vector'''

    def __init__(self, word_embeds, pool='max'):
        '''
        Required input: word embeddings stored in dict structure available for look-up
        pool: sentence embeddings to be obtained either via average pooling ('average') or max pooing ('max') from word embeddings. Default is average pooling.
        '''
        self.word_embeds = word_embeds
        self.pool_method = pool

    def transform(self, X, **transform_params):
        '''
        Transformation function: X is list of sentence/tweet - strings in the train data. Returns list of embeddings, each embedding representing one tweet
        '''
        return [self.get_sent_embedding(sent, self.word_embeds, self.pool_method) for sent in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_sent_embedding(self, sentence, word_embeds, pool):
        '''
        Obtains sentence embedding representing a whole sentence / tweet
        '''
        # simply get dim of embeddings
        l_vector = len(word_embeds['sport'])

        # replace each word in sentence with its embedding representation via look up in the embedding dict strcuture
        # if no word_embedding available for a word, just ignore the word
        # [[0.234234,-0.276583...][0.2343, -0.7356354, 0.123 ...][0.2344356, 0.12477...]...]
        list_of_embeddings = [word_embeds[word.lower()] for word in sentence.split() if word.lower() in word_embeds]

	    # Obtain sentence embeddings either by average or max pooling on word embeddings of the sentence
        # Option via argument 'pool'
        if pool == 'pool':
            sent_embedding = [sum(col) / float(len(col)) for col in zip(*list_of_embeddings)]  # average pooling
        elif pool == 'max':
            sent_embedding = [max(col) for col in zip(*list_of_embeddings)]	# max pooling
        else:
            raise ValueError('Unknown pooling method!')

        # Handle small sentences not in word embedding
        if len(sent_embedding) != l_vector:
            sent_embedding = [0] * l_vector

        return sent_embedding

# Hate Speed dataset - Italian 

In [7]:
print('Reading EVALITA FB and TW training data')
Xtrain = [] 
Ytrain = []
Xtrain,Ytrain = get_haspeed_file(FB_train, Xtrain, Ytrain)
Xtrain,Ytrain = get_haspeed_file(TW_train, Xtrain, Ytrain)
Xtrain = cleanup_italian_text(Xtrain)

Xtest = [] 
Ytest = []
Xtest,Ytest = get_haspeed_file(FB_test, Xtest, Ytest)
Xtest,Ytest = get_haspeed_file(TW_test, Xtest, Ytest)
Xtest = cleanup_italian_text(Xtest)

Reading EVALITA FB and TW training data


## Use embedding: HateSpeech Evalita

In [10]:
print('Loading pretrained word embeddings from Merenda 2018')
embeddings_ita1, vocab = get_embedding(embedding_ita1_file)

print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(embeddings_ita1, pool='pool'))])

clf = LinearSVC()
classifier_ita1 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_ita1.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_ita1.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_ita1.predict(Xtest)
get_metrics(Ytest, Yguess)

Loading pretrained word embeddings from [file name article]
Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 0.9991500212494687

Predicting classifier of test data
precision: 0.821656050955414
Recall:    0.7732267732267732
F1-score:  0.7967061245496655


## Evaluate gender bias

In [12]:
bias_test_df = pd.read_csv(bias_samples_ita)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_ita1.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.930379746835443
Recall:    0.441
F1-score:  0.5983717774762551


Metrics related to male dataset
precision: 0.9868421052631579
Recall:    0.45
F1-score:  0.618131868131868


Metrics related to female dataset
precision: 0.8780487804878049
Recall:    0.432
F1-score:  0.5790884718498659


Gender bias metrics
 False Positive Equality Difference (FPED): 0.054
 False Negative Equality Difference (FNED): 0.017999999999999905


## Use Embeddings: Generic Evalita

In [13]:
print('Loading pretrained word embeddings from [file name article]')
embeddings_ita2 = get_embedding_sqlite(embedding_ita2_file)

print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(embeddings_ita2, pool='pool'))])

clf = LinearSVC()
classifier_ita2 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_ita2.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_ita2.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_ita2.predict(Xtest)
get_metrics(Ytest, Yguess)

Loading pretrained word embeddings from [file name article]
Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 0.9991500212494687

Predicting classifier of test data
precision: 0.8118393234672304
Recall:    0.7672327672327672
F1-score:  0.7889060092449922


## Evaluate gender bias

In [14]:
bias_test_df = pd.read_csv(bias_samples_ita)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_ita2.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.9842696629213483
Recall:    0.438
F1-score:  0.6062283737024221


Metrics related to male dataset
precision: 0.9914893617021276
Recall:    0.466
F1-score:  0.6340136054421769


Metrics related to female dataset
precision: 0.9761904761904762
Recall:    0.41
F1-score:  0.5774647887323944


Gender bias metrics
 False Positive Equality Difference (FPED): 0.006
 False Negative Equality Difference (FNED): 0.05599999999999994


## Use embedding: word2vec

In [13]:
print('Loading pretrained word embeddings from Berardi 2015')
embeddings_ita3 = get_embedding2(embedding_ita3_file)

print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(embeddings_ita3, pool='pool'))])

clf = LinearSVC()
classifier_ita3 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_ita3.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_ita3.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_ita3.predict(Xtest)
get_metrics(Ytest, Yguess)

Loading pretrained word embeddings from Berardi 2015
Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 0.9991496598639457

Predicting classifier of test data
precision: 0.8167202572347267
Recall:    0.7612387612387612
F1-score:  0.7880041365046536


## Evaluate gender bias

In [14]:
bias_test_df = pd.read_csv(bias_samples_ita)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_ita3.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.8597972972972973
Recall:    0.509
F1-score:  0.6394472361809045


Metrics related to male dataset
precision: 0.9217081850533808
Recall:    0.518
F1-score:  0.6632522407170295


Metrics related to female dataset
precision: 0.8038585209003215
Recall:    0.5
F1-score:  0.6165228113440198


Gender bias metrics
 False Positive Equality Difference (FPED): 0.078
 False Negative Equality Difference (FNED): 0.018000000000000016


# StormfrontWS dataset - English 

In [10]:
print('Reading StormfrontWS Web forum data')
Xtrain, Ytrain = get_StormfrontWS_files(forum_post_eng, 'train')
Xtrain = cleanup_english_text(Xtrain)

Xtest, Ytest = get_StormfrontWS_files(forum_post_eng, 'test')
Xtest = cleanup_english_text(Xtest)

Reading StormfrontWS Web forum data


## Embedding word2vec Google News

In [14]:
model_w2v_AP = load_wordvec_model(embedding_eng1_file, True)

In [15]:
print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(model_w2v_AP, pool='pool'))])

clf = LinearSVC()
classifier_eng1 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_eng1.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_eng1.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_eng1.predict(Xtest)
get_metrics(Ytest, Yguess)

Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 1.0

Predicting classifier of test data
precision: 0.7451737451737451
Recall:    0.8075313807531381
F1-score:  0.7751004016064258


## Evaluate gender bias

In [17]:
bias_test_df = pd.read_csv(bias_samples_eng)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_eng1.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.8819277108433735
Recall:    0.366
F1-score:  0.5173144876325089


Metrics related to male dataset
precision: 0.9281437125748503
Recall:    0.31
F1-score:  0.46476761619190404


Metrics related to female dataset
precision: 0.8508064516129032
Recall:    0.422
F1-score:  0.5641711229946524


Gender bias metrics
 False Positive Equality Difference (FPED): 0.049999999999999996
 False Negative Equality Difference (FNED): 0.11199999999999999


## Embedding GloVe

In [8]:
glove_6B = load_glove_embeddings(embedding_eng2_file)

In [11]:
print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(glove_6B, pool='pool'))])

clf = LinearSVC()
classifier_eng2 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_eng2.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_eng2.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_eng2.predict(Xtest)
get_metrics(Ytest, Yguess)

Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 1.0

Predicting classifier of test data
precision: 0.7244094488188977
Recall:    0.7698744769874477
F1-score:  0.7464503042596349


## Evaluate gender bias

In [12]:
bias_test_df = pd.read_csv(bias_samples_eng)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_eng2.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.7566462167689162
Recall:    0.37
F1-score:  0.49697783747481533


Metrics related to male dataset
precision: 0.7958115183246073
Recall:    0.304
F1-score:  0.43994211287988416


Metrics related to female dataset
precision: 0.7315436241610739
Recall:    0.436
F1-score:  0.5463659147869674


Gender bias metrics
 False Positive Equality Difference (FPED): 0.082
 False Negative Equality Difference (FNED): 0.132


#  
# Tweet Hate Speech - Italian 

In [14]:
X, Y = get_HSC_file(HS_text)

test_i = set()
while len(test_i) < 100: 
    test_i.add(int(np.random.uniform(len(X))))

Xtrain = []
Ytrain = []
Xtest = []
Ytest = []
for i in range(len(X)): 
    if i in list(test_i): 
        Xtest.append(X[i])
        Ytest.append(Y[i])
    else: 
        Xtrain.append(X[i])
        Ytrain.append(Y[i])
print('Train set ration hate not hate {} over {}'.format(sum([int(y) for y in Ytrain])/len(Ytrain), len(Xtrain)))
print('Test set ration hate not hate {} over {}'.format(sum([int(y) for y in Ytest])/len(Ytest), len(Xtest)))

Train set ration hate not hate 0.14929577464788732 over 1420
Test set ration hate not hate 0.13 over 100


In [20]:
embeddings_ita2 = get_embedding_sqlite(embedding_ita2_file)

print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(embeddings_ita2, pool='pool'))])

clf = LinearSVC()
classifier_ita2 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_ita2.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_ita2.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_ita2.predict(Xtest)
get_metrics(Ytest, Yguess)

Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 1.0

Predicting classifier of test data
precision: 0.5
Recall:    0.15384615384615385
F1-score:  0.23529411764705882


## Evaluate gender bias

In [21]:
bias_test_df = pd.read_csv(bias_samples_ita)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_ita2.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 1.0
Recall:    0.006
F1-score:  0.011928429423459244


Metrics related to male dataset
precision: 1.0
Recall:    0.004
F1-score:  0.00796812749003984


Metrics related to female dataset
precision: 1.0
Recall:    0.008
F1-score:  0.015873015873015872


Gender bias metrics
 False Positive Equality Difference (FPED): 0.0
 False Negative Equality Difference (FNED): 0.0040000000000000036


#  
# Tweet Hate Speech - English

In [9]:
X, Y = get_davidson_file(davidson_eng)

test_i = set()
while len(test_i) < 3000: 
    test_i.add(int(np.random.uniform(len(X))))

Xtrain = []
Ytrain = []
Xtest = []
Ytest = []
for i in range(len(X)): 
    if i in list(test_i): 
        Xtest.append(X[i])
        Ytest.append(Y[i])
    else: 
        Xtrain.append(X[i])
        Ytrain.append(Y[i])
print('Train set ration hate not hate {} over {}'.format(sum([int(y) for y in Ytrain])/len(Ytrain), len(Xtrain)))
print('Test set ration hate not hate {} over {}'.format(sum([int(y) for y in Ytest])/len(Ytest), len(Xtest)))

Train set ration hate not hate 0.832484047192765 over 21783
Test set ration hate not hate 0.8286666666666667 over 3000


In [10]:
model_w2v_AP = load_wordvec_model(embedding_eng1_file, True)

print('Preparing vectorizer and classifier')
count_word = TfidfVectorizer(analyzer='word', ngram_range=(1,3), binary=False, sublinear_tf=False)
count_char = TfidfVectorizer(analyzer='char', ngram_range=(2,4), binary=False, sublinear_tf=False)

vectorizer = FeatureUnion([('word', count_word), ('char', count_char), 
                           ('word_embeds', Embeddings(model_w2v_AP, pool='pool'))])

clf = LinearSVC()
classifier_ita2 = Pipeline([('vectorize', vectorizer), ('classify', clf)])

print('Fitting on training data')
classifier_ita2.fit(Xtrain, Ytrain)

print('Predicting classifier of train data')
Yguess = classifier_ita2.predict(Xtrain)
scores = metrics.precision_recall_fscore_support(Ytrain, Yguess, average='binary', pos_label='1')
print('F1-score: {}'.format(scores[2]))

print('\nPredicting classifier of test data')
Yguess = classifier_ita2.predict(Xtest)
get_metrics(Ytest, Yguess)

Preparing vectorizer and classifier
Fitting on training data
Predicting classifier of train data
F1-score: 1.0

Predicting classifier of test data
precision: 0.9719213798636182
Recall:    0.9746580852775543
F1-score:  0.9732878087969471


## Evaluate gender bias

In [11]:
bias_test_df = pd.read_csv(bias_samples_ita)
bias_test_df['correct'] = ['0' if label == 'NOT_BAD' else '1' for label in bias_test_df['hate']]
X = cleanup_italian_text(bias_test_df['text'])
bias_test_df['pred'] = classifier_ita2.predict(X)

get_gender_bias_metrics(bias_test_df)

Metrics related to full dataset
precision: 0.46411483253588515
Recall:    0.194
F1-score:  0.2736248236953455


Metrics related to male dataset
precision: 0.45136186770428016
Recall:    0.232
F1-score:  0.30647291941875826


Metrics related to female dataset
precision: 0.484472049689441
Recall:    0.156
F1-score:  0.2360060514372163


Gender bias metrics
 False Positive Equality Difference (FPED): 0.11599999999999996
 False Negative Equality Difference (FNED): 0.07599999999999996
